#EJERCICIO 5  

En  el  caso  de  las  reseñas  realizadas por usuarios, utiliza análisis de sentimientos con 
modelos pre entrenados y guarda la clasificación predecida de cada reseña. 
 
Luego,  crea  un  sistema  de  búsquedas  por similitud semántica y que permita filtrar por 
sentimiento para obtener.

In [4]:
#Abrimos el archivo con las reseñas y guardamos cada comentario en una lista
with open('./datos/informacion/sagrada_comments.txt', 'r', encoding='utf-8') as file:
    comments = file.read()

In [5]:
# Separamos los comentarios por el delimitador específico que se utiliza en el archivo
lista_comentarios = comments.split('\n\n;\n\n')

In [6]:
len(lista_comentarios)

2501

In [10]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

# Cargamos el tokenizador y el modelo
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# Creamos un pipeline de clasificación
nlp = pipeline("sentiment-analysis", truncation= True, model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [11]:
# Obtenemos las predicciones de sentimiento para cada frase y lo guardamos en una lista
sentimiento_comentarios = []
for frase in lista_comentarios:
    sentimiento_comentarios.append(nlp(frase))

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [21]:
from sentence_transformers import SentenceTransformer, util

In [ ]:
# Cargamos el modelo preentrenado multilingüe para generar embeddings de oraciones
modelo_cased_v2 = SentenceTransformer('distiluse-base-multilingual-cased-v2')

In [14]:
#Procesamos las oraciones
embeddings = modelo_cased_v2.encode(lista_comentarios, convert_to_tensor=True)

In [30]:
def buscar_reseñas(query, estrellas, top_n=3):
    # Codificamos la consulta para buscar reseñas
    query_embedding = modelo_cased_v2.encode(query, convert_to_tensor=True)
    reseñas_filtradas = []
    # Filtramos las reseñas por el número de estrellas
    for i, sentimiento in enumerate(sentimiento_comentarios):
        if sentimiento[0]['label'] == estrellas:
            # Calculamos la similitud entre la consulta y las reseñas
            similarity = util.cos_sim(query_embedding, embeddings[i])
            # Guardamos la reseña y su similitud
            reseñas_filtradas.append((lista_comentarios[i], similarity))
            #print(f"Reseña: {lista_comentarios[i]}\nSimilitud: {similarity.item()}\n")
    # Ordenamos las reseñas por similitud y seleccionamos las mejores
    reseñas_filtradas.sort(key=lambda x: x[1], reverse=True)
    return reseñas_filtradas[:top_n]
            

In [65]:
resultados = buscar_reseñas("it is boring", "3 stars", top_n=3)

In [66]:
for reseña, similitud in resultados:
    print(f"Reseña: {reseña}\nSimilitud: {similitud.item()}\n")

Reseña: nice.. but boring
Similitud: 0.5766628980636597

Reseña: It's shiny but kinda boring.
Similitud: 0.5170140266418457

Reseña: ok but a little bit boring for me
Similitud: 0.45535808801651

